# 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助您：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(agent101)`，即可执行

In [1]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate agent101

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "agent101" ]; then
    echo "✅ 当前单元格已成功激活到 agent101 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(agent101)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate agent101"
fi

echo "=========================================" 

== Conda 环境检查报告 (仅针对当前 Bash 子进��) ==
✅ 当前单元格已成功激活到 agent101 环境。
✅ 正在使用的环境路径: /root/miniconda3/envs/agent101

💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter��核。
   如果需要后续单元格也使用此环境，请执行以下操作:
   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。


In [2]:
# 2. 设置pip 为清华源
%pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config 

Writing to /root/.config/pip/pip.conf
Note: you may need to restart the kernel to use updated packages.
ERROR: Need an action (debug, edit, get, list, set, unset) to perform.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT

env: HF_ENDPOINT=https://hf-mirror.com
https://hf-mirror.com


In [4]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助您：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                               |
|:-------------|:-----------------------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                                           |
| CPU 信息     | 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz (1 physical cores, 2 logical cores) |
| 内存信息     | 3.78 GB (Available: 2.27 GB)                                                       |
| GPU 信息     | No GPU found (nvidia-smi not found)                                                |
| CUDA 信息    | CUDA not found                                                                     |
| Python 版本  | 3.10.18                                                                            |
| Conda 版本   | conda 24.4.0                                         

# 教程使用指南

本教程**需要API密钥**进行交互。如果您没有API密钥，可以通过[OpenAI Platform](https://platform.openai.com/)注册获取。如果您想使用DeepSeek模型，也可以通过[DeepSeek Platform](https://platform.deepseek.com/)获取API密钥。

## 如何开始

1. 将此仓库克隆到本地机器，按照Python、Conda和Jupyter。

2. 浏览器访问jupyter后，通过运行以下命令安装所需的依赖项：
 

In [5]:
# 安装OpenAI Python SDK
%pip install openai==1.107.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


3. 设置您的API密钥和模型名称。将`"OPENAI_API_KEY"`替换为您实际的OpenAI API密钥。

In [6]:
import os, getpass

def _set_env(var: str):
    """
    安全地设置环境变量
    如果环境变量不存在，则提示用户输入

    Args:
        var (str): 要设置的环境变量名称
    """
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# 设置 OpenAI API代理地址 (例如：https://api.apiyi.com/v1）
_set_env("OPENAI_BASE_URL")
# 设置OpenAI API密钥; OpenAI API密钥获取 (https://api.apiyi.com）
_set_env("OPENAI_API_KEY")

4. 按顺序运行notebook单元格，遵循提供的说明。

---

## 使用说明和提示 💡

- 本课程使用GPT-4o，温度设置为0。我们将在课程后面详细讨论温度参数。现在，只需理解这些设置能产生更确定性的结果。本课程中的所有提示工程技术也适用于其他OpenAI模型，如Qwen，以及DeepSeek等模型。

- 您可以使用`Shift + Enter`执行单元格并移动到下一个。

- 当您到达教程页面底部时，导航到文件夹中的下一个编号文件，或者如果您已完成该章节文件内的内容，则转到下一个编号文件夹。

### OpenAI SDK和Chat Completions API
我们将在整个教程中使用[OpenAI Python SDK](https://platform.openai.com/docs/libraries)和[Chat Completions API](https://platform.openai.com/docs/api-reference/chat)。

下面是本教程中运行提示的示例。首先，我们创建`get_completion`，这是一个辅助函数，用于向GPT发送提示并返回GPT生成的响应。现在运行该单元格。

现在我们将为GPT编写一个示例提示，并通过运行我们的`get_completion`辅助函数来打印GPT的输出。运行下面的单元格将在其下方打印出GPT的响应。

请随意修改提示字符串，以获得GPT的不同响应。

In [7]:
# 提示文本
prompt = "如何写好提示词"

# 获取GPT回答
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-5",
  messages=[
    {"role": "system", "content": "你是一个有用的大模型应用编程助手"},
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='写好提示词的关键是：把目标说清楚，把上下文补齐，把输出和质量标准定死。下面给你一套可复用的方法、模板和示例。\n\n一、核心原则\n- 明确目标：用一句话说明你要的结果和目的。\n- 指定角色与受众：让模型扮演谁（专家、工程师、编辑），面向谁（新人、CIO、开发者）。\n- 给足背景与范围：提供必要上下文、数据来源、时间范围、边界条件。\n- 定义约束：长度、语气、风格、技术版本、禁止项（如不得编造、不得使用某库）。\n- 规定输出格式：例如“按要点列出”或“返回可解析的JSON，且不含额外文字”。\n- 要过程但不强求“思维链”：可要求“分步骤解决并简要说明关键依据”，避免让模型展开详细内心推理。\n- 要质量控制：要求自检、引用来源、标注不确定性、给出验证或测试。\n- 允许澄清：让模型在信息不足时先提3–5个针对性问题。\n- 少而精：避免冗长，去除与任务无关的指令。\n\n二、通用提示词模板（可直接填空）\n- 角色：你是……\n- 目标：请完成……\n- 背景：已知……\n- 输入：将收到的输入是……\n- 范围与边界：仅限于……，不涉及……\n- 约束：语气……，长度……，风格……，技术版本……，禁止……\n- 输出格式：返回……（例如：标题+要点；或JSON字段：title, steps, risks）\n- 过程与依据：分步骤完成，并简要列出关键依据/判断点（不需展开完整思考过程）\n- 质量检查：若信息不足请先提问；若存在不确定性请标注；必要时给出来源或验证方式\n- 交互偏好：如需澄清，先提最多N个问题再继续\n\n三、针对不同任务的模板\n1) 知识问答\n- 问题：……\n- 范围：仅基于……（如官方文档/标准）\n- 时间或版本：截至……；如信息可能过时请说明\n- 来源要求：至少列出2个可核验来源（标题+链接或出版信息）\n- 不确定性：如无法确定，请说明原因并给出下一步建议\n- 输出格式：结论→要点→来源列表\n\n2) 编程与代码\n- 语言与版本：如 Python 3.11 / Node.js 18\n- 目标：实现……函数/模块\n- I/O 规范：输入……；输出……\n- 约束：时间/空间复杂度目标、允许/禁止的库、风格（PEP8）、错误处理策略\n- 测试：提供至

In [8]:
# 提取消息对象
message_content = completion.choices[0].message.content
message_role = completion.choices[0].message.role
# 格式化打印输出
print("--- 模型回复 ---")
print(f"角色 (Role): {message_role}")
print("----------------")
print(f"内容 (Content):\n{message_content}")
print("----------------")

--- 模型回复 ---
角色 (Role): assistant
----------------
内容 (Content):
写好提示词的关键是：把目标说清楚，把上下文补齐，把输出和质量标准定死。下面给你一套可复用的方法、模板和示例。

一、核心原则
- 明确目标：用一句话说明你要的结果和目的。
- 指定角色与受众：让模型扮演谁（专家、工程师、编辑），面向谁（新人、CIO、开发者）。
- 给足背景与范围：提供必要上下文、数据来源、时间范围、边界条件。
- 定义约束：长度、语气、风格、技术版本、禁止项（如不得编造、不得使用某库）。
- 规定输出格式：例如“按要点列出”或“返回可解析的JSON，且不含额外文字”。
- 要过程但不强求“思维链”：可要求“分步骤解决并简要说明关键依据”，避免让模型展开详细内心推理。
- 要质量控制：要求自检、引用来源、标注不确定性、给出验证或测试。
- 允许澄清：让模型在信息不足时先提3–5个针对性问题。
- 少而精：避免冗长，去除与任务无关的指令。

二、通用提示词模板（可直接填空）
- 角色：你是……
- 目标：请完成……
- 背景：已知……
- 输入：将收到的输入是……
- 范围与边界：仅限于……，不涉及……
- 约束：语气……，长度……，风格……，技术版本……，禁止……
- 输出格式：返回……（例如：标题+要点；或JSON字段：title, steps, risks）
- 过程与依据：分步骤完成，并简要列出关键依据/判断点（不需展开完整思考过程）
- 质量检查：若信息不足请先提问；若存在不确定性请标注；必要时给出来源或验证方式
- 交互偏好：如需澄清，先提最多N个问题再继续

三、针对不同任务的模板
1) 知识问答
- 问题：……
- 范围：仅基于……（如官方文档/标准）
- 时间或版本：截至……；如信息可能过时请说明
- 来源要求：至少列出2个可核验来源（标题+链接或出版信息）
- 不确定性：如无法确定，请说明原因并给出下一步建议
- 输出格式：结论→要点→来源列表

2) 编程与代码
- 语言与版本：如 Python 3.11 / Node.js 18
- 目标：实现……函数/模块
- I/O 规范：输入……；输出……
- 约束：时间/空间复杂度目标、允许/禁止的库、风格（PEP8）、错误处理策略
- 测试：提供至少3个测试

---

## OpenAI 聊天 API 消息角色对比

| 角色 (`role`) | 作用和目的 | 示例（用于 `content`） | 代码中的对应 |
| :--- | :--- | :--- | :--- |
| **`system`** （系统） | **设置 AI 的全局行为、人设和约束**。这条消息指导模型在整个对话中应采取的态度和风格。 | "你是一位幽默且专业的营销专家。" | 实际应用中，它**应替代**您使用的 `developer` 角色。 |
| **`user`** （用户） | **代表人类用户发出的问题或指令**。这是模型需要直接回复的主要内容。 | "帮我写一个关于新产品的三点宣传口号。" | **`"role": "user"`**，内容为 `prompt` 变量。 |
| **`assistant`** （助手/模型） | **代表 AI 模型自身的回复**。用于在 `messages` 列表中提供历史对话记录，帮助模型记住上下文。 | "当然，这是您的三个口号：[口号内容]" | 在实际 API 响应中会生成。 |
| **`tool`** （工具） | **提供外部工具或函数执行后的结果**。模型根据此结果来生成最终的用户回复。 | `{"status": "success", "data": "今天的实时天气是 25°C。"}` | 用于复杂工作流，如函数调用（Function Calling）。 |
| **`developer`** （开发者） | **设置模型初始指令**。历史上曾用于特定模型系列（如 `o1`），但在 GPT-4o 等新模型中，它**功能上等同于 `system`**。 | "你是一个有用的助手" | **`"role": "developer"`**，内容为 "你是一个有用的助手"。 |



#### OpenAI API 配置封装详解

🎯 主要功能：
- 多源配置管理：环境变量 → IPython存储 → 默认值
- 一键环境设置：自动配置OpenAI客户端和便捷函数
- 安全密钥处理：脱敏显示、占位符检测
- 智能环境检测：支持Jupyter notebook和普通Python环境

🔧 核心特性：
- 配置优先级：环境变量 > IPython存储 > 默认值
- 自动依赖安装：检测并安装缺失的依赖包
- 优雅错误处理：详细的配置指导和错误提示
- 模块化设计：每个函数职责单一，易于维护和扩展

🚀 使用场景：
- 学习环境：Jupyter notebook交互式配置
- 开发环境：.env文件本地配置
- 生产环境：环境变量安全配置
- 代理服务：支持国内API代理服务

📋 支持的配置方式：
1. 环境变量：OPENAI_API_KEY, OPENAI_MODEL, OPENAI_BASE_URL
2. .env文件：项目根目录或模块目录的.env文件
3. IPython存储：%store API_KEY, %store MODEL_NAME
4. 默认值：gpt-4o模型作为兜底配置




In [9]:
# 🔧 OpenAI API 配置方式

# 方式一：环境变量（推荐）
# 在终端中设置，或者在 .env 文件中设置
# export OPENAI_API_KEY="sk-your-api-key-here"
# export MODEL_NAME="gpt-4o"
# export OPENAI_BASE_URL="https://vip.apiyi.com/v1"  # 可选：国内代理地址

# 方式二：在此处直接设置（仅用于测试）
# API_KEY = "sk-your-api-key-here"  # 请替换为您的实际OpenAI API密钥
# MODEL_NAME = "gpt-4o"  # GPT-4o模型（推荐）或 "deepseek-r1"

# 将API_KEY和MODEL_NAME变量存储在IPython存储中，以便在不同notebook中使用
# %store API_KEY
# %store MODEL_NAME

# 配置验证和自动设置
from config import setup_notebook_environment, print_config_info, validate_config

# 如果还没设置过API密钥，请先配置
print("📋 配置检查中...")
if not validate_config():
    print("""
⚠️  需要配置API密钥！请选择以下方式之一：

🌟 方式一：环境变量（推荐）
# 在终端中运行：
export OPENAI_API_KEY="sk-your-api-key-here"

# 或者使用项目根目录的脚本：
./set_bashrc_env.sh OPENAI_API_KEY "sk-your-api-key-here"

🔧 方式二：在notebook中设置
API_KEY = "sk-your-api-key-here"
MODEL_NAME = "gpt-4o"
%store API_KEY
%store MODEL_NAME

📥 获取API密钥：
- OpenAI官方: https://platform.openai.com/api-keys
- DeepSeek: https://platform.deepseek.com/api-keys  
- 国内代理: https://www.apiyi.com/register/?aff_code=we80
    """)
else:
    print("✅ API配置已就绪")
    print_config_info()

📋 配置检查中...
✅ 配置验证通过
✅ API配置已就绪
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://api.apiyi.com/v1
  🤖 模型: deepseek-chat
  🌐 API地址: https://api.apiyi.com/v1
  🔑 API密钥: sk-AGnnT...A14f



In [10]:
# 🚀 自动设置OpenAI环境
from config import setup_notebook_environment

# 设置OpenAI客户端和get_completion函数
# 此函数会自动处理环境变量和IPython存储的配置
client, get_completion = setup_notebook_environment()

# 现在您可以直接使用 get_completion 函数了！
print("🎉 环境设置完成！可以开始使用 get_completion 函数了。")

✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://api.apiyi.com/v1
  🤖 模型: deepseek-chat
  🌐 API地址: https://api.apiyi.com/v1
  🔑 API密钥: sk-AGnnT...A14f

🎉 环境设置完成！可以开始使用 get_completion 函数了。


In [11]:
# 提示文本
prompt = "如何写好提示词"

# 获取GPT回答
print(get_completion(prompt))

写好提示词（Prompt）是有效使用AI工具（如ChatGPT、MidJourney等）的关键。好的提示词能清晰传达需求，提高输出质量。以下是撰写高质量提示词的实用技巧：

---

### **1. 明确目标**
- **核心原则**：先想清楚你想要什么（答案、创意、数据、风格等）。
- 例子：  
  ❌ 模糊："写一篇关于健康的文章"  
  ✅ 明确："写一篇针对30-40岁人群的800字科普文章，主题是‘如何通过饮食预防心血管疾病’，要求包含科学依据和实用建议"

---

### **2. 结构化表达**
- **背景 + 任务 + 细节 + 输出格式**  
  例：  
  "你是一名经验丰富的市场营销专家（背景），请分析新能源汽车行业2023年的趋势（任务），重点对比特斯拉和比亚迪的策略（细节），用分点列表形式输出（格式）"

---

### **3. 关键细节**
- **具体参数**：  
  - 长度（如"500字"）、风格（如"幽默风趣"、"学术严谨"）  
  - 受众（如"面向小学生解释"）  
  - 限制条件（如"避免使用专业术语"）  
- 例子：  
  ❌ "生成一张狗的图片"  
  ✅ "生成一张金毛犬在阳光下的草坪上奔跑的摄影风格图片，4K高清，浅景深"

---

### **4. 避免歧义**
- 多用精确词汇，少用抽象词。  
  例：  
  ❌ "写点有趣的东西"  
  ✅ "写一个关于程序员与咖啡机的冷笑话，不超过100字"

---

### **5. 分步引导（复杂任务）**
- 对于多环节任务，拆解步骤：  
  "第一步：列出5个关于AI伦理的争议点；第二步：针对每个争议点各写一段分析；第三步：总结成一份报告"

---

### **6. 示例参考**
- **提供范例**能快速对齐预期：  
  "模仿以下广告文案风格：『把1000首歌装进口袋——iPod』，为我们的新产品‘智能水杯’设计3条类似 slogan"

---

### **7. 迭代优化**
- **逐步修正**：根据首次结果调整提示词。  
  例：  
  第一轮输出太笼统 → 追加提示："请增加具体案例和数据支持"

---

### **8. 不同场景的提示词技巧**
- **创作类**：指定风格、情感、参照对象  
 